# NIA Core Agent Testing

This notebook provides interactive testing of core agents to verify their functionality and interactions.

In [ ]:
import sys
sys.path.append('..')

from src.nia.agents.base import TinyPerson
from src.nia.memory.two_layer import TwoLayerMemorySystem
from src.nia.nova.orchestrator import Nova

# Import core agents
from src.nia.agents.specialized.parsing_agent import ParsingAgent
from src.nia.agents.specialized.belief_agent import BeliefAgent
from src.nia.agents.specialized.emotion_agent import EmotionAgent
from src.nia.agents.specialized.reflection_agent import ReflectionAgent
from src.nia.agents.specialized.desire_agent import DesireAgent

## 1. Memory System Setup

Initialize the two-layer memory system with Neo4j and vector store.

In [ ]:
# Initialize memory system
memory = TwoLayerMemorySystem(
    neo4j_uri="bolt://localhost:7687",
    neo4j_user="neo4j",
    neo4j_password="password"
)

# Test memory operations
await memory.store_with_domain(
    content="This is a test memory",
    domain="personal"
)

# Query test
result = await memory.query_by_domain(
    query="test memory",
    domain="personal"
)
print("Memory Query Result:", result)

## 2. Core Agent Testing

Test each core agent's functionality individually.

In [ ]:
# Initialize agents
parsing_agent = ParsingAgent()
belief_agent = BeliefAgent()
emotion_agent = EmotionAgent()
reflection_agent = ReflectionAgent()
desire_agent = DesireAgent()

# Test parsing agent
test_text = "I need to create a documentation site for the project."
parsed_result = await parsing_agent.parse_text(test_text)
print("Parsing Result:", parsed_result)

## 3. Belief and Emotion Processing

Test belief formation and emotional response processing.

In [ ]:
# Test belief formation
test_evidence = "The project needs better documentation based on user feedback."
belief_result = await belief_agent.process_evidence(test_evidence)
print("Belief Formation:", belief_result)

# Test emotional response
emotion_result = await emotion_agent.process_input(test_evidence)
print("Emotional Analysis:", emotion_result)

## 4. Reflection and Desire Processing

Test reflection capabilities and desire formation.

In [ ]:
# Test reflection
reflection_input = {
    "current_state": "Documentation needs improvement",
    "past_actions": ["Created initial docs", "Received feedback"],
    "outcomes": ["Users found docs unclear"]
}
reflection_result = await reflection_agent.reflect(reflection_input)
print("Reflection Result:", reflection_result)

# Test desire formation
desire_result = await desire_agent.process_motivation(
    context="Documentation improvement needed",
    priority="high"
)
print("Desire Formation:", desire_result)

## 5. Nova Orchestration

Test Nova's ability to coordinate multiple agents.

In [ ]:
# Initialize Nova
nova = Nova(memory_system=memory)

# Register agents
nova.register_agent(parsing_agent)
nova.register_agent(belief_agent)
nova.register_agent(emotion_agent)
nova.register_agent(reflection_agent)
nova.register_agent(desire_agent)

# Test multi-agent coordination
test_input = "We should improve the documentation based on user feedback."
result = await nova.process_input(test_input)
print("Nova Coordination Result:", result)

## 6. Emergent Task Detection

Test the system's ability to detect and handle emergent tasks.

In [ ]:
# Test emergent task detection
conversation = [
    "The documentation needs to be more comprehensive.",
    "We should add more code examples.",
    "Users are requesting better API documentation."
]

for message in conversation:
    result = await nova.process_conversation(message)
    print(f"\nMessage: {message}")
    print("Processing Result:", result)
    
# Check emergent tasks
emergent_tasks = nova.get_emergent_tasks()
print("\nEmergent Tasks:", emergent_tasks)

## 7. Memory Integration

Test how agents interact with the memory system.

In [ ]:
# Test memory integration
test_interaction = "User feedback suggests adding more code examples to docs."

# Store in memory
await memory.store_with_domain(test_interaction, "work")

# Process with agents
parsed = await parsing_agent.parse_text(test_interaction)
beliefs = await belief_agent.process_evidence(test_interaction)
reflection = await reflection_agent.reflect({
    "current_state": test_interaction,
    "past_actions": [],
    "outcomes": []
})

print("Memory Integration Results:")
print("Parsed:", parsed)
print("Beliefs:", beliefs)
print("Reflection:", reflection)

## 8. Domain Awareness

Test how agents handle different domains (personal vs. work).

In [ ]:
# Test domain handling
personal_input = "I should review my learning progress."
work_input = "The team needs to update the API documentation."

# Process both domains
personal_result = await nova.process_input(personal_input, domain="personal")
work_result = await nova.process_input(work_input, domain="work")

print("Personal Domain Result:", personal_result)
print("Work Domain Result:", work_result)

## 9. Error Handling

Test how the system handles various error conditions.

In [ ]:
# Test error handling
try:
    # Test with invalid domain
    result = await nova.process_input("Test message", domain="invalid")
except ValueError as e:
    print("Domain Error:", e)

try:
    # Test with missing memory connection
    memory.neo4j_store = None
    result = await nova.process_input("Test message")
except Exception as e:
    print("Memory Error:", e)

# Restore memory connection
memory = TwoLayerMemorySystem(
    neo4j_uri="bolt://localhost:7687",
    neo4j_user="neo4j",
    neo4j_password="password"
)